In [1]:
import pandas as pd
import numpy as np
import spacy
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import gensim.downloader as api

In [2]:
df = pd.read_csv("/content/draft_50000.csv")
df = df.rename(columns = {'text': 'data'}, inplace = False)
df = df.rename(columns = {'class': 'Label'}, inplace = False)
df.head()

,Unnamed: 0,data,Label
0,51297,get crush guy definitely way old laugh literal...,0.0
1,24705,go to july 2018i hope 8 month enough change mind,1.0
2,185969,want live anymore 23 right thinking end life p...,1.0
3,201675,every time get period want kill already depres...,1.0
4,52701,story incomplete similar story op case dad spe...,1.0


In [3]:
df['data'] = df['data'].str.lower().str.replace('[^\w\s]', '', regex=True)
df.dropna(subset=['data', 'Label'], inplace=True)

In [4]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

In [5]:
t = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [6]:
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() for word in doc]

    # Removing stop words and punctuations
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # Return preprocessed list of tokens
    return mytokens

In [7]:
df['data1'] = df['data'].apply(spacy_tokenizer)

In [8]:
df['vec'] = df['data1'].apply(lambda x: np.mean([t[token] for token in x if token in t] or [np.zeros(t.vector_size)], axis=0))

In [9]:
X = np.vstack(df['vec'])  # Convert list of arrays to a matrix
y = df['Label']

In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM model
svm_model = SVC(kernel='linear', C=1, random_state=42)
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:",accuracy)
print("\nClassification Report:\n", classification_rep)

Accuracy: 0.872093023255814

Classification Report:
               precision    recall  f1-score   support

         0.0       0.89      0.85      0.87       516
         1.0       0.85      0.90      0.88       516

    accuracy                           0.87      1032
   macro avg       0.87      0.87      0.87      1032
weighted avg       0.87      0.87      0.87      1032

